In [1]:
# Import necessary libraries
import os
from google.colab import drive

In [ ]:
# Install necessary packages
!pip install openai
!pip install langchain
!pip install pymupdf
!pip install chromadb


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11


In [4]:
!pip install langchain-chroma

In [5]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# File paths for the annual reports in Google Drive
files = {
    "2018": "./Walmart_Form_10-K_01-31-2018.pdf",
    "2023": "./walmart-inc-2023-annual-report.pdf"
}


In [ ]:
# Import necessary modules after installation
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI

In [ ]:
# Set OpenAI API Key
import openai
# os.environ["OPENAI_API_KEY"] = "XXX"

In [ ]:
# Initialize OpenAI Chat LLM and embeddings
llm = ChatOpenAI(model="gpt-4", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # Specify embedding model

In [ ]:
# Define text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Reduce chunk size for smaller token input
    chunk_overlap=200,
    add_start_index=True
)

In [ ]:
# Function to extract relevant sections
def extract_sections(file_path, section_keywords):
    loader = PyMuPDFLoader(file_path)
    pages = loader.load()
    relevant_texts = []
    for page in pages:
        content = page.page_content.lower()
        if any(keyword.lower() in content for keyword in section_keywords):
            relevant_texts.append(page.page_content)
    return "\n".join(relevant_texts)


In [ ]:
# Extract MD&A and Risk Factors sections
sections = {
    year: {
        "MD&A": extract_sections(path, ["Management's Discussion and Analysis"]),
        "Risk Factors": extract_sections(path, ["Risk Factors"])
    }
    for year, path in files.items()
}

In [ ]:
# Summarize sections to reduce size
def summarize_text(text, section_name, year):
    prompt = f"Summarize the following {section_name} section from the {year} annual report:\n\n{text}"
    summary = llm.predict(prompt)
    return summary

In [ ]:
# Summarize MD&A and Risk Factors
summarized_sections = {
    year: {
        "MD&A": summarize_text(sections[year]["MD&A"], "MD&A", year),
        "Risk Factors": summarize_text(sections[year]["Risk Factors"], "Risk Factors", year),
    }
    for year in sections
}

In [ ]:
# Create vector store
documents = []
for year in summarized_sections:
    documents.append(Document(page_content=summarized_sections[year]["MD&A"], metadata={"year": year, "type": "MD&A"}))
    documents.append(Document(page_content=summarized_sections[year]["Risk Factors"], metadata={"year": year, "type": "Risk Factors"}))

vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)


In [ ]:
# Define comparison template
COMPARISON_TEMPLATE = """
You are an assistant tasked with comparing two documents. Below are two sections extracted from annual reports of a Fortune 500 company from different years.

Section 1 (Year {year_1}):
{doc_1}

Section 2 (Year {year_2}):
{doc_2}

Please provide a detailed comparison of the two sections. Highlight:
1. Changes in business outlook and opportunities.
2. Changes in the risks identified by the firm.
3. Overall evolution of the firm’s priorities and strategies.
"""

comparison_prompt = ChatPromptTemplate.from_template(COMPARISON_TEMPLATE)

In [ ]:
# QA chain setup
def create_qa_chain():
    return (
        {
            "doc_1": RunnablePassthrough(),
            "doc_2": RunnablePassthrough(),
            "year_1": RunnablePassthrough(),
            "year_2": RunnablePassthrough()
        }
        | comparison_prompt
        | llm
        | StrOutputParser()
    )

qa_chain = create_qa_chain()


In [ ]:
# Perform comparison
for section in ["MD&A", "Risk Factors"]:
    question = {
        "doc_1": summarized_sections["2018"][section],
        "doc_2": summarized_sections["2023"][section],
        "year_1": "2018",
        "year_2": "2023"
    }
    print(f"Comparison for {section} section:\n")
    print(qa_chain.invoke(question))